# 01 - Construct loans repayments history features

## What is this notebook?

This notebook creates the dataset to create features that is about loan repayments history. For that, we have 2 columns that are stored in `database.db` file that are in `\databases\database.db`:

#### Loans Table
- **id (int)**: Unique identifier for the loan.
- **user_id (int)**: Unique identifier for the user who has taken the loan.
- **amount (float)**: The amount of loan disbursed.
- **total_amount (float)**: The amount of loan, including fees.
- **due_amount (int)**: The amount of the loan by the due date if there are no repayments during the contract period. Good to get interest rates.
- **due_date (object)**: The date by which the loan is due.
- **status (object)**: Current status of the loan (e.g., repaid, debt_collection, ongoing, debt_repaid).
    - **repaid**: A loan that was was paid until due.
    - **debt_collection**: A loan that was not paid until due.
    - **debt_repaid**: A loan that was not paid until due but we recovered the money somehow.
    - **cancelled**: A canceled loan.
    - **error**: Operational error.
- **created_at (object)**: Timestamp of when the loan record was created. <u>Have it as the beginning of the loan</u>.

#### Loan Repayments Table
- **id (int)**: Unique identifier for the repayment record.
- **loan_id (int)**: The ID of the loan this repayment is associated with.
- **type (object)**: The type of repayment (e.g., autopilot, pix).
- **amount (float)**: The amount that was repaid.
- **status (object)**: Status of the repayment.
    - **paid**: A loan repayment that effectively happened.
    - **defaulted**: A loan repayment that didn`t come to reality.
    - **refunded**: A loan repayment that happened but was fully refunded to the user. 
- **created_at (object)**: Timestamp of when the repayment record was created. <u>Have it as the moment of the repayment</u>.


## Results
The final dataset have this new features:
 - `amt_paid_till_first_month`: Calculates for a loan the amount paid in the first month.
 - `amt_paid_till_sec_month`: Calculates for a loan the amount paid till the second month.
 - `amt_paid_till_trd_month`: Calculates for a loan the amount paid till the third month.
 - `amt_paid_first_month`: Calculates for a loan the amount paid in the first month.
 - `amt_paid_sec_month`: Calculates for a loan the amount paid in the second month.
 - `amt_paid_third_month`: Calculates for a loan the amount paid in the third month.
 - `pct_paid_first_month`: Calculates the percentage of a loan the amount paid in the first month.
 - `pct_paid_sec_month`: Calculates the percentage of a loan the amount paid in the second month.
 - `pct_paid_third_month`: Calculates the percentage of a loan the amount paid in the third month.
 - `amt_defaulted_till_first_month`: Calculates for a loan the amount defaulted till the first month.
 - `amt_defaulted_till_sec_month`: Calculates for a loan the amount defaulted till the second month.
 - `amt_defaulted_till_trd_month`: Calculates for a loan the amount defaulted till the third month.
 - `amt_defaulted_first_month`: Calculates for a loan the amount defaulted in the first month.
 - `amt_defaulted_sec_month`: Calculates for a loan the amount defaulted in the second month.
 - `amt_defaulted_third_months`: Calculates for a loan the amount defaulted in the third month.
 - `most_frequent_repayment_type_for_loan`: Calculates the most frequent repayment type for a loan.

 This dataset is located in `./data/processed/loans_repayment_history_features.csv'` 

## 1 - Imports

In [1]:
import os 
os.chdir("../../")

In [2]:
import sqlalchemy
import pandas as pd 
import numpy as np

from pandas.tseries.offsets import DateOffset
from datetime import datetime,date
from typing import Union

## 2 - Read tables

In [3]:
engine = sqlalchemy.create_engine("sqlite:///./database/database.db", echo=True)

df_loans = pd.read_sql(
    sql="""
    SELECT * FROM loans l
    """,
    con=engine
)
df_loans_repay = pd.read_sql(
    sql="""
    SELECT * FROM loan_repayments lr
    """,
    con=engine
)

df_transactions = pd.read_sql(
    sql="""
    SELECT * FROM transactions tr
    """,
    con=engine
)

2024-04-07 15:08:58,138 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-07 15:08:58,139 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("
    SELECT * FROM loans l
    ")
2024-04-07 15:08:58,140 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-07 15:08:58,141 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("
    SELECT * FROM loans l
    ")
2024-04-07 15:08:58,143 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-07 15:08:58,145 INFO sqlalchemy.engine.Engine 
    SELECT * FROM loans l
    
2024-04-07 15:08:58,146 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-07 15:08:58,386 INFO sqlalchemy.engine.Engine ROLLBACK
2024-04-07 15:08:58,388 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-07 15:08:58,388 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("
    SELECT * FROM loan_repayments lr
    ")
2024-04-07 15:08:58,389 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-07 15:08:58,390 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("
    SELECT * FR

In [4]:
# seeing loans table
print("Shape df_loans:",df_loans.shape)
df_loans.head()

Shape df_loans: (6746, 8)


,id,user_id,amount,total_amount,due_amount,due_date,status,created_at
0,0,3070,6000.0,6045.28,6459000000,2022-05-02,repaid,2022-02-01 00:47:29.575000+00:00
1,1,2546,6000.0,6045.28,6459000000,2022-05-02,repaid,2022-02-01 00:49:51.763000+00:00
2,2,2413,6000.0,6045.28,6459000000,2022-05-02,repaid,2022-02-01 01:24:40.537000+00:00
3,3,2585,6000.0,6045.28,6459000000,2022-05-02,debt_collection,2022-02-01 02:52:59.803000+00:00
4,4,2556,6000.0,6045.28,6459000000,2022-05-02,repaid,2022-02-01 02:53:07.123000+00:00


In [5]:
df_loans.dtypes

id                int64
user_id           int64
amount          float64
total_amount    float64
due_amount        int64
due_date         object
status           object
created_at       object
dtype: object

In [6]:
# check the categorical variables in df_loans
df_loans["status"].unique()

array(['repaid', 'debt_collection', 'debt_repaid', 'error', 'cancelled'],
      dtype=object)

In [7]:
# seeing loan_repayments table
print("Shape df_loan_repayments:",df_loans_repay.shape)
df_loans_repay.head()

Shape df_loan_repayments: (172445, 6)


,id,loan_id,type,amount,status,created_at
0,1,2,autopilot,269.7,paid,2022-02-01 00:10:08.057000+00:00
1,2,4,autopilot,2550.0,paid,2022-02-01 00:10:08.102000+00:00
2,3,53,pix,1500.0,defaulted,2022-02-01 13:09:53.757000+00:00
3,4,22,autopilot,630.0,paid,2022-02-02 01:21:34.278000+00:00
4,5,70,autopilot,120.0,paid,2022-02-02 01:21:34.449000+00:00


In [8]:
df_loans_repay.dtypes

id              int64
loan_id         int64
type           object
amount        float64
status         object
created_at     object
dtype: object

In [9]:
# check the categorical tables
print("status unique values:",df_loans_repay["status"].unique())
print("type unique values:",df_loans_repay["type"].unique())

status unique values: ['paid' 'defaulted' 'refunded']
type unique values: ['autopilot' 'pix']


In [10]:
# seeing loan_repayments table
print("Shape df_transactions:",df_transactions.shape)
df_transactions.head()

Shape df_transactions: (1448684, 9)


,id,user_id,amount,status,capture_method,payment_method,installments,card_brand,created_at
0,1,2546,449.6,approved,emv,credit,1,mastercard,2021-12-02 13:41:53.548000+00:00
1,2,2546,269.8,approved,emv,credit,2,mastercard,2021-12-02 15:57:58.742000+00:00
2,3,2546,149.9,approved,contactless,debit,1,visa,2021-12-02 19:36:31.859000+00:00
3,4,2546,142.0,approved,contactless,credit,1,visa,2021-12-02 19:37:11.442000+00:00
4,5,2546,156.0,approved,emv,debit,1,mastercard,2021-12-02 20:20:26.648000+00:00


In [11]:
df_transactions.dtypes

id                  int64
user_id             int64
amount            float64
status             object
capture_method     object
payment_method     object
installments        int64
card_brand         object
created_at         object
dtype: object

In [12]:
# check the categorical tables
print("card_brand unique values:",df_transactions["card_brand"].unique())
print("payment method unique values:",df_transactions["payment_method"].unique())
print("capture method unique values:",df_transactions["capture_method"].unique())
print("status unique values:",df_transactions["status"].unique())

card_brand unique values: ['mastercard' 'visa' 'elo' 'hipercard' 'amex']
payment method unique values: ['credit' 'debit']
capture method unique values: ['emv' 'contactless' 'payment_link' 'ecommerce' 'mpos' 'payment_link_web']
status unique values: ['approved' 'denied']


## 2 - Exploratory data analysis
Let's make a simple check of the data of our tables. To understand what we have

In [13]:
## check the amount of dates we have

## df_loans check dates
year_month_loans_created = pd.Series([f"{d.date().month}/{d.date().year}" for d in pd.to_datetime(df_loans["created_at"],format="ISO8601")])
year_month_loans_created.value_counts(ascending=False)

4/2022     1490
7/2022      957
2/2022      897
6/2022      778
8/2022      728
9/2022      723
3/2022      589
5/2022      538
10/2022      46
Name: count, dtype: int64

In [14]:
## df_loans_repayment check dates
year_month_loans_repay_created = pd.Series([f"{d.date().month}/{d.date().year}" for d in pd.to_datetime(df_loans_repay["created_at"],format="ISO8601")])
year_month_loans_repay_created.value_counts(ascending=False)

7/2022     25762
5/2022     24785
6/2022     23826
8/2022     21728
9/2022     21267
10/2022    15747
4/2022     14172
3/2022      9939
11/2022     7095
2/2022      5606
12/2022     2415
1/2023        94
3/2023         8
6/2023         1
Name: count, dtype: int64

In [15]:
## df_transactions check dates
year_month_trans_created = pd.Series([f"{d.date().month}/{d.date().year}" for d in pd.to_datetime(df_transactions["created_at"],format="ISO8601")])
year_month_trans_created.value_counts(ascending=False)

5/2022     162907
7/2022     159536
6/2022     155262
4/2022     154135
3/2022     144825
8/2022     142071
9/2022     123364
10/2022    112900
11/2022     83245
2/2022      68964
12/2022     55315
1/2022      50641
12/2021     32049
1/2023       3470
Name: count, dtype: int64

In [16]:
## let's see the time to get the loan repaid is after three months that the loan was created.
df_loans_months_to_due_date = df_loans[["due_date","created_at"]]
df_loans_months_to_due_date.loc[:,"due_date"] = pd.to_datetime(df_loans["due_date"],format="ISO8601")
df_loans_months_to_due_date.loc[:,"created_at"] = pd.to_datetime(df_loans["created_at"],format="ISO8601")
df_loans_months_to_due_date.loc[:,"days_between"] = [
    (dates[0].date() - dates[1].date()).days for dates in df_loans_months_to_due_date.values
]
df_loans_months_to_due_date["days_between"].unique()

/tmp/ipykernel_173718/78303140.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_loans_months_to_due_date.loc[:,"days_between"] = [


array([ 90,  91,  96,  95,  94,  93,  92, 112,  97])

Simple overall:
- We can see that most of the loans are by 90 days after the loan was created.
- Also for dates, we see that loans have all dates from Fev/2022 till Oct/2022, transactions have dates from Dez/2021 till Jan/2023 and loans repayments have data from Jan/2022 till Jun/2023.

## 3 - Preprocessing Data

In [17]:
#convert to datetime
df_loans["created_at"] = pd.to_datetime(df_loans["created_at"],utc=True,format="ISO8601")
df_loans_repay["created_at"] = pd.to_datetime(df_loans_repay["created_at"],utc=True,format="ISO8601")
df_transactions["created_at"] = pd.to_datetime(df_transactions["created_at"],utc=True,format="ISO8601")

#convert to date
df_loans["due_date"] = pd.to_datetime(df_loans["due_date"],format="%Y-%m-%d")

#create date_created 
df_loans["date_created"] = df_loans["created_at"].apply(func=lambda d:d.date())
df_loans_repay["date_created"] = df_loans_repay["created_at"].apply(func=lambda d:d.date())
df_transactions["date_created"] = df_transactions["created_at"].apply(func=lambda d:d.date())

## 3 - Created the repaied transactions features

### 3.1 - Functions to create features from loans_repayments table

In [18]:
def create_month_offset(add_value:Union[int,None]=None,
                        sub_value:Union[int,None]=None,
                        date:datetime=datetime(1970,1,1))-> datetime:
    """
    Create a new datetime reference based on add value or sub value. If
    is add_value then a month is added based on this value else if is a sub_value
    then a month is subtract based on this value.

    Example:
    >> dt = datetime(2022,1,1)
    >> new_dt = create_month_offset(add_value=1,date=dt)
    >> print(new_dt)
    2022-02-01 00:00:00
    
    >> dt2 = datetime(2022,1,1)
    >> new_dt2 = create_month_offset(sub_value=1,date=dt2)
    >> print(new_dt2)
    2022-01-01 00:00:00

    Args:
        add_value (int,optional): Month frequency to add. Must be positive. Defaults to None.
        sub_value (int,optional): Month frequency to subtract. Must be negative. Defaults to None.
        date (datetime): Datetime to calcuate the date offset.

    Returns:
        datetime: A new datetime based in the month offset. 
    """ 
    if add_value and not sub_value:
        month_offset = DateOffset(months=add_value)
    elif sub_value and not add_value:
        month_offset = DateOffset(months=sub_value)
    return date+month_offset

def create_month_col_offset(dataframe:pd.DataFrame,
                            month_frequency:int,
                            date_ref_col:str,
                            new_col_name:str)->pd.DataFrame:
    """
    Create a new column that creates a month_offset for every value
    in date_ref_col. If month_frequency is positive then a month is added 
    or if month_frequency is negative then a month is subtract.

    Example:
    >> dfr = pd.DataFrame({"pay_date":[datetime(2022,3,4),datetime(2022,4,3)]})
    >> dfr = create_month_col_offset(dataframe=dfr,
                              month_frequency=1,
                              date_ref_col="pay_date",
                              new_col_name="1_month_after_pay_date")
    >> dfr
        | pay_date   | 1_month_after_pay_date
        | 2022-03-04 |      2022-04-04
        | 2022-04-03 |      2022-05-04

     >> df_check_due_dates = pd.DataFrame({"due_date":[datetime(2023,5,4),datetime(2023,7,4)]})
     >> df_check_due_dates = create_month_col_offset(dataframe=df_check_due_dates,
                              month_frequency=-2,
                              date_ref_col="due_date",
                              new_col_name="2_month_before_due_date")
     >> df_check_due_dates
        | due_date   | 2_month_before_due_date
        | 2023-05-04 |      2022-04-04
        | 2022-07-04 |      2022-05-04

    Args:
        dataframe (pd.DataFrame): The input dataframe to create the new datetime column.
        month_frequency (int): Month frequency to add or subtract. To add, this value must
        be positive. To subtract, then this value must be negative.
        date_ref_col (str): The date reference column to create the month offset.
        new_col_name (str): The name of the new column to added.

    Returns:
        pd.DataFrame: A new dataframe with a new datetime column added with name defined by variable
        new_col_name. For every value in this column a new date have a new month_frequency added or
        subtract.
    """    
    
    new_df = dataframe.copy()
    if month_frequency>0:
        new_df[new_col_name] = dataframe[date_ref_col].apply(func=lambda d: create_month_offset(add_value=month_frequency,date=d))
        return new_df
    elif month_frequency<0:
        new_df[new_col_name] = dataframe[date_ref_col].apply(func=lambda d: create_month_offset(sub_value=month_frequency,date=d))
        return new_df


df_pay_ranges = df_loans[["date_created","due_date"]].drop_duplicates(ignore_index=True)

df_pay_ranges = df_pay_ranges.pipe(func=create_month_col_offset,
                                   month_frequency=1,
                                   date_ref_col="date_created",
                                   new_col_name="first_month_pay_date")\
                             .pipe(create_month_col_offset,
                                   month_frequency=1,
                                   date_ref_col="first_month_pay_date",
                                   new_col_name="sec_month_pay_date"
                             )
df_pay_ranges

,date_created,due_date,first_month_pay_date,sec_month_pay_date
0,2022-02-01,2022-05-02,2022-03-01,2022-04-01
1,2022-02-01,2022-05-03,2022-03-01,2022-04-01
2,2022-02-02,2022-05-03,2022-03-02,2022-04-02
3,2022-02-02,2022-05-09,2022-03-02,2022-04-02
4,2022-02-02,2022-05-04,2022-03-02,2022-04-02
...,...,...,...,...
253,2022-09-29,2022-12-28,2022-10-29,2022-11-29
254,2022-09-30,2022-12-29,2022-10-30,2022-11-30
255,2022-10-01,2022-12-30,2022-11-01,2022-12-01
256,2022-10-02,2022-12-31,2022-11-02,2022-12-02


In [19]:
def filter_amt_repayments_by_dates_per_loan_id(
        dataframe_loans:pd.DataFrame,
        dataframe_repayments:pd.DataFrame,
        date_ref_col_df_loans:str,
        date_ref_col_df_repayments:str,
        begin_dates:Union[list,np.array],
        end_dates:Union[list,np.array]
)->list:
    """
    This function union the loans dataframe and dataframe repayments to select the 
    loans repayments that happended after the day the loan was created (begin_date) but before a date (end_date).
    To select the repayments, these steps are made for each begin_date and end_date in the begin_dates and end_dates
    lists:
    1. Creates the pairs of (begin_date, end_date) to considere. It's made by the order they are
    on the lists. So, the i-th date in begin_dates is made a pair with the respective j-th date in end_dates. This
    means that the first value in begin_dates make a pair the first value in end_dates, then, the second value in
    begin_dates make a pair with the second value in end_dates and so on.
    2. For each pair of (begin_date,end_date), selects the loans in the loans dataframe that 
    were created for each begin_date.
    3. For each pair of (begin_date, end_date), filters the loans repayments by selecting
    only the repayments that are from the loans that were selected in the step 2 and
    filters the repayments that happended after the begin_date and before the end_date.

    At the end, this filtered dataframe is added to a list to save the selected repayments for each
    begin_date and end_date in begin_dates and end_dates list.

    Args:
        dataframe_loans (pd.DataFrame): Dataframe that contains all loans created.
        dataframe_repayments (pd.DataFrame): Dataframe with all repayments of a loan that happened after
        a loan is made.
        date_ref_col_df_loans (str): Date column to use as reference from the dataframe_loans.
        date_ref_col_df_repayments (str): Date column to use as reference from the dataframe_loans.
        begin_dates (Union[list,np.array]): Initial dates to considerer to select the loans that were
        created in that day and filter the loan repayments that happended after these dates.
        end_dates (Union[list,np.array]): Initial dates to considerer to filter the dataframe_repayments to
        select the loan repayments that happended before these dates.

    Returns:
        list: A list of dataframes of all loan repayments that happended after the 
        loan was created (begin_date) and before the end_date for each pair (begin_date,end_date) created 
        by the begin_dates and end_dates lists.
    """    
    dfs = []
    for b_date,e_date in zip(begin_dates,end_dates):
        idxs_loans = dataframe_loans[dataframe_loans[date_ref_col_df_loans]==b_date]["id"].to_list()
        filter_repayments = dataframe_repayments[dataframe_repayments["loan_id"].isin(idxs_loans)]
        filter_df = filter_repayments[(filter_repayments[date_ref_col_df_repayments]>b_date)
                              & (filter_repayments[date_ref_col_df_repayments]<=e_date)]
        dfs.append(filter_df)

    return dfs

def calculate_amt_loan_paid_per_id(
        dataframe:pd.DataFrame,
        new_column_name:str
)->pd.DataFrame:
    """
    Calculates the sum of amount paied for each loan_id.

    Args:
        dataframe (pd.DataFrame): Initial dataframe to considere to make
        the group by and calcute the amount.
        new_column_name (str): Name of the sum column created by the group by.

    Returns:
        pd.DataFrame: A new dataframe that have the "index" and "new_column_name"
        columns that indicates the sum of amount paied for each loan_id.
    """    
    new_df = dataframe.groupby(by="loan_id")["amount"].sum()
    return new_df.to_frame(name=new_column_name).reset_index()

In [20]:
## create reference time variables 
first_month_begin_dates = df_pay_ranges["date_created"].to_list()
first_month_end_dates = [d.date() for d in df_pay_ranges["first_month_pay_date"].to_list()]

sec_month_end_dates = [d.date() for d in df_pay_ranges["sec_month_pay_date"].to_list()]

trd_month_end_dates = [d.date() for d in df_pay_ranges["due_date"].to_list()]

### 3.2 - Create amount repayment features for paid repayments

In [21]:
## Define dataset of paided loans
paid_loans = df_loans_repay[df_loans_repay["status"]=="paid"]

print("Number of paied_loans:",paid_loans.shape[0])

Number of paied_loans: 162108


In [22]:
dfs_first_month_repaid_loans = filter_amt_repayments_by_dates_per_loan_id(
    dataframe_repayments=paid_loans,
    dataframe_loans=df_loans,
    date_ref_col_df_loans="date_created",
    date_ref_col_df_repayments="date_created",
    begin_dates=first_month_begin_dates,
    end_dates=first_month_end_dates
)

dfs_sec_month_repaid_loans = filter_amt_repayments_by_dates_per_loan_id(
    dataframe_loans=df_loans,
    dataframe_repayments=paid_loans,
    date_ref_col_df_loans="date_created",
    date_ref_col_df_repayments="date_created",
    begin_dates=first_month_begin_dates,
    end_dates=sec_month_end_dates
)

dfs_trd_month_repaid_loans = filter_amt_repayments_by_dates_per_loan_id(
    dataframe_loans=df_loans,
    dataframe_repayments=paid_loans,
    date_ref_col_df_loans="date_created",
    date_ref_col_df_repayments="date_created",
    begin_dates=first_month_begin_dates,
    end_dates=trd_month_end_dates
)

In [23]:
amt_paid_loans_first_month = [
    calculate_amt_loan_paid_per_id(
        dataframe=df,
        new_column_name="amt_paid_till_first_month"
    )
    for df in dfs_first_month_repaid_loans
]

amt_paid_loans_sec_month = [
    calculate_amt_loan_paid_per_id(
        dataframe=df,
        new_column_name="amt_paid_till_sec_month"
    )
    for df in dfs_sec_month_repaid_loans
]

amt_paid_loans_trd_month = [
    calculate_amt_loan_paid_per_id(
        dataframe=df,
        new_column_name="amt_paid_till_trd_month"
    )
    for df in dfs_trd_month_repaid_loans
]

In [24]:
df_all_amt_paid_first_month = pd.concat(amt_paid_loans_first_month).drop_duplicates(subset=["loan_id"],keep="first")
df_all_amt_paid_sec_month = pd.concat(amt_paid_loans_sec_month).drop_duplicates(subset=["loan_id"],keep="first")
df_all_amt_paid_trd_month = pd.concat(amt_paid_loans_trd_month).drop_duplicates(subset=["loan_id"],keep="first")

In [25]:
df_paid_amt_loans = df_all_amt_paid_first_month.merge(
                          right=df_all_amt_paid_sec_month,
                          on=["loan_id"],
                          how="outer")\
                        .merge(
                          right=df_all_amt_paid_trd_month,
                          on=["loan_id"],
                          how="outer"
                        )
df_paid_amt_loans

,loan_id,amt_paid_till_first_month,amt_paid_till_sec_month,amt_paid_till_trd_month
0,0,3619.35,6190.95,6190.95
1,1,1928.84,4545.53,6244.28
2,2,2145.75,2145.75,6032.88
3,3,42.00,162.00,162.00
4,4,3521.92,3521.92,3521.92
...,...,...,...,...
6521,6741,2591.71,2591.71,2591.71
6522,6742,6456.88,6456.88,6456.88
6523,6743,5443.63,6388.02,6388.02
6524,6744,1443.60,1936.60,3045.40


In [26]:
# create amt per month cols 
df_paid_amt_loans["amt_paid_first_month"] = df_paid_amt_loans["amt_paid_till_first_month"].copy()
df_paid_amt_loans["amt_paid_sec_month"] = df_paid_amt_loans["amt_paid_till_sec_month"] - df_paid_amt_loans["amt_paid_till_first_month"]
df_paid_amt_loans["amt_paid_third_month"] = df_paid_amt_loans["amt_paid_till_trd_month"] - df_paid_amt_loans["amt_paid_till_sec_month"]
df_paid_amt_loans

,loan_id,amt_paid_till_first_month,amt_paid_till_sec_month,amt_paid_till_trd_month,amt_paid_first_month,amt_paid_sec_month,amt_paid_third_month
0,0,3619.35,6190.95,6190.95,3619.35,2571.60,0.00
1,1,1928.84,4545.53,6244.28,1928.84,2616.69,1698.75
2,2,2145.75,2145.75,6032.88,2145.75,0.00,3887.13
3,3,42.00,162.00,162.00,42.00,120.00,0.00
4,4,3521.92,3521.92,3521.92,3521.92,0.00,0.00
...,...,...,...,...,...,...,...
6521,6741,2591.71,2591.71,2591.71,2591.71,0.00,0.00
6522,6742,6456.88,6456.88,6456.88,6456.88,0.00,0.00
6523,6743,5443.63,6388.02,6388.02,5443.63,944.39,0.00
6524,6744,1443.60,1936.60,3045.40,1443.60,493.00,1108.80


In [27]:
df_loans_with_repay_feats = df_loans.merge(
    right=df_paid_amt_loans,
    right_on="loan_id",
    left_on="id",
    how="left"
)
df_loans_with_repay_feats

,id,user_id,amount,total_amount,due_amount,due_date,status,created_at,date_created,loan_id,amt_paid_till_first_month,amt_paid_till_sec_month,amt_paid_till_trd_month,amt_paid_first_month,amt_paid_sec_month,amt_paid_third_month
0,0,3070,6000.0,6045.28,6459000000,2022-05-02,repaid,2022-02-01 00:47:29.575000+00:00,2022-02-01,0.0,3619.35,6190.95,6190.95,3619.35,2571.60,0.00
1,1,2546,6000.0,6045.28,6459000000,2022-05-02,repaid,2022-02-01 00:49:51.763000+00:00,2022-02-01,1.0,1928.84,4545.53,6244.28,1928.84,2616.69,1698.75
2,2,2413,6000.0,6045.28,6459000000,2022-05-02,repaid,2022-02-01 01:24:40.537000+00:00,2022-02-01,2.0,2145.75,2145.75,6032.88,2145.75,0.00,3887.13
3,3,2585,6000.0,6045.28,6459000000,2022-05-02,debt_collection,2022-02-01 02:52:59.803000+00:00,2022-02-01,3.0,42.00,162.00,162.00,42.00,120.00,0.00
4,4,2556,6000.0,6045.28,6459000000,2022-05-02,repaid,2022-02-01 02:53:07.123000+00:00,2022-02-01,4.0,3521.92,3521.92,3521.92,3521.92,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6741,6741,2130,2500.0,2518.87,3228820000,2023-01-01,repaid,2022-10-03 20:44:11.967000+00:00,2022-10-03,6741.0,2591.71,2591.71,2591.71,2591.71,0.00,0.00
6742,6742,549,6000.0,6045.28,7749160000,2023-01-01,repaid,2022-10-03 21:03:02.995000+00:00,2022-10-03,6742.0,6456.88,6456.88,6456.88,6456.88,0.00,0.00
6743,6743,1414,6000.0,6045.28,7749160000,2023-01-01,repaid,2022-10-03 21:33:47.964000+00:00,2022-10-03,6743.0,5443.63,6388.02,6388.02,5443.63,944.39,0.00
6744,6744,2070,6000.0,6045.28,7749160000,2023-01-01,debt_repaid,2022-10-03 22:29:03.256000+00:00,2022-10-03,6744.0,1443.60,1936.60,3045.40,1443.60,493.00,1108.80


In [28]:
# create pct_features 
df_loans_with_repay_feats["pct_paid_first_month"] = df_loans_with_repay_feats["amt_paid_till_first_month"]/df_loans_with_repay_feats["total_amount"]
df_loans_with_repay_feats["pct_paid_sec_month"] = df_loans_with_repay_feats["amt_paid_till_sec_month"]/df_loans_with_repay_feats["total_amount"]
df_loans_with_repay_feats["pct_paid_third_month"] = df_loans_with_repay_feats["amt_paid_till_trd_month"]/df_loans_with_repay_feats["total_amount"]
df_loans_with_repay_feats

,id,user_id,amount,total_amount,due_amount,due_date,status,created_at,date_created,loan_id,amt_paid_till_first_month,amt_paid_till_sec_month,amt_paid_till_trd_month,amt_paid_first_month,amt_paid_sec_month,amt_paid_third_month,pct_paid_first_month,pct_paid_sec_month,pct_paid_third_month
0,0,3070,6000.0,6045.28,6459000000,2022-05-02,repaid,2022-02-01 00:47:29.575000+00:00,2022-02-01,0.0,3619.35,6190.95,6190.95,3619.35,2571.60,0.00,0.598707,1.024096,1.024096
1,1,2546,6000.0,6045.28,6459000000,2022-05-02,repaid,2022-02-01 00:49:51.763000+00:00,2022-02-01,1.0,1928.84,4545.53,6244.28,1928.84,2616.69,1698.75,0.319065,0.751914,1.032918
2,2,2413,6000.0,6045.28,6459000000,2022-05-02,repaid,2022-02-01 01:24:40.537000+00:00,2022-02-01,2.0,2145.75,2145.75,6032.88,2145.75,0.00,3887.13,0.354946,0.354946,0.997949
3,3,2585,6000.0,6045.28,6459000000,2022-05-02,debt_collection,2022-02-01 02:52:59.803000+00:00,2022-02-01,3.0,42.00,162.00,162.00,42.00,120.00,0.00,0.006948,0.026798,0.026798
4,4,2556,6000.0,6045.28,6459000000,2022-05-02,repaid,2022-02-01 02:53:07.123000+00:00,2022-02-01,4.0,3521.92,3521.92,3521.92,3521.92,0.00,0.00,0.582590,0.582590,0.582590
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6741,6741,2130,2500.0,2518.87,3228820000,2023-01-01,repaid,2022-10-03 20:44:11.967000+00:00,2022-10-03,6741.0,2591.71,2591.71,2591.71,2591.71,0.00,0.00,1.028918,1.028918,1.028918
6742,6742,549,6000.0,6045.28,7749160000,2023-01-01,repaid,2022-10-03 21:03:02.995000+00:00,2022-10-03,6742.0,6456.88,6456.88,6456.88,6456.88,0.00,0.00,1.068086,1.068086,1.068086
6743,6743,1414,6000.0,6045.28,7749160000,2023-01-01,repaid,2022-10-03 21:33:47.964000+00:00,2022-10-03,6743.0,5443.63,6388.02,6388.02,5443.63,944.39,0.00,0.900476,1.056695,1.056695
6744,6744,2070,6000.0,6045.28,7749160000,2023-01-01,debt_repaid,2022-10-03 22:29:03.256000+00:00,2022-10-03,6744.0,1443.60,1936.60,3045.40,1443.60,493.00,1108.80,0.238798,0.320349,0.503765


### 3.3 - Create the amount repayment features from defaulted repayments


In [29]:
defaulted_loans = df_loans_repay[df_loans_repay["status"]=="defaulted"]
print("Number of defaulted_loans:",defaulted_loans.shape[0])

Number of defaulted_loans: 8740


In [30]:
dfs_first_month_defaulted_loans = filter_amt_repayments_by_dates_per_loan_id(
    dataframe_repayments=defaulted_loans,
    dataframe_loans=df_loans,
    date_ref_col_df_loans="date_created",
    date_ref_col_df_repayments="date_created",
    begin_dates=first_month_begin_dates,
    end_dates=first_month_end_dates
)

dfs_sec_month_defaulted_loans = filter_amt_repayments_by_dates_per_loan_id(
    dataframe_loans=df_loans,
    dataframe_repayments=defaulted_loans,
    date_ref_col_df_loans="date_created",
    date_ref_col_df_repayments="date_created",
    begin_dates=first_month_begin_dates,
    end_dates=sec_month_end_dates
)

dfs_trd_month_defaulted_loans = filter_amt_repayments_by_dates_per_loan_id(
    dataframe_loans=df_loans,
    dataframe_repayments=defaulted_loans,
    date_ref_col_df_loans="date_created",
    date_ref_col_df_repayments="date_created",
    begin_dates=first_month_begin_dates,
    end_dates=trd_month_end_dates
)

In [31]:
amt_defaulted_loans_first_month = [
    calculate_amt_loan_paid_per_id(
        dataframe=df,
        new_column_name="amt_defaulted_till_first_month"
    )
    for df in dfs_first_month_defaulted_loans
]

amt_defaulted_loans_sec_month = [
    calculate_amt_loan_paid_per_id(
        dataframe=df,
        new_column_name="amt_defaulted_till_sec_month"
    )
    for df in dfs_sec_month_defaulted_loans
]

amt_defaulted_loans_trd_month = [
    calculate_amt_loan_paid_per_id(
        dataframe=df,
        new_column_name="amt_defaulted_till_trd_month"
    )
    for df in dfs_trd_month_defaulted_loans
]

In [32]:
df_all_amt_defaulted_first_month = pd.concat(amt_defaulted_loans_first_month).drop_duplicates(subset=["loan_id"],keep="first")
df_all_amt_defaulted_sec_month = pd.concat(amt_defaulted_loans_sec_month).drop_duplicates(subset=["loan_id"],keep="first")
df_all_amt_defaulted_trd_month = pd.concat(amt_defaulted_loans_trd_month).drop_duplicates(subset=["loan_id"],keep="first")

In [33]:
df_defaulted_amt_loans = df_all_amt_defaulted_first_month.merge(
                          right=df_all_amt_defaulted_sec_month,
                          on=["loan_id"],
                          how="outer")\
                        .merge(
                          right=df_all_amt_defaulted_trd_month,
                          on=["loan_id"],
                          how="outer"
                        )
df_defaulted_amt_loans

,loan_id,amt_defaulted_till_first_month,amt_defaulted_till_sec_month,amt_defaulted_till_trd_month
0,4,3055.39,3055.39,3055.39
1,5,403.02,403.02,403.02
2,7,8651.19,8651.19,8651.19
3,8,12200.36,12200.36,12200.36
4,11,725.06,725.06,725.06
...,...,...,...,...
3675,6740,NaN,25.50,25.50
3676,6741,200.00,200.00,200.00
3677,6743,1663.37,1663.37,1663.37
3678,6744,16.59,16.59,16.59


In [34]:
# create amt per month cols 
df_defaulted_amt_loans["amt_defaulted_first_month"] = df_defaulted_amt_loans["amt_defaulted_till_first_month"].copy()
df_defaulted_amt_loans["amt_defaulted_sec_month"] = df_defaulted_amt_loans["amt_defaulted_till_sec_month"] - df_defaulted_amt_loans["amt_defaulted_till_first_month"]
df_defaulted_amt_loans["amt_defaulted_third_month"] = df_defaulted_amt_loans["amt_defaulted_till_trd_month"] - df_defaulted_amt_loans["amt_defaulted_till_sec_month"]
df_defaulted_amt_loans

,loan_id,amt_defaulted_till_first_month,amt_defaulted_till_sec_month,amt_defaulted_till_trd_month,amt_defaulted_first_month,amt_defaulted_sec_month,amt_defaulted_third_month
0,4,3055.39,3055.39,3055.39,3055.39,0.0,0.0
1,5,403.02,403.02,403.02,403.02,0.0,0.0
2,7,8651.19,8651.19,8651.19,8651.19,0.0,0.0
3,8,12200.36,12200.36,12200.36,12200.36,0.0,0.0
4,11,725.06,725.06,725.06,725.06,0.0,0.0
...,...,...,...,...,...,...,...
3675,6740,NaN,25.50,25.50,NaN,NaN,0.0
3676,6741,200.00,200.00,200.00,200.00,0.0,0.0
3677,6743,1663.37,1663.37,1663.37,1663.37,0.0,0.0
3678,6744,16.59,16.59,16.59,16.59,0.0,0.0


In [35]:
df_loans_with_repay_feats = df_loans_with_repay_feats.merge(
    right=df_defaulted_amt_loans,
    right_on="loan_id",
    left_on="loan_id",
    how="left"
)
df_loans_with_repay_feats

,id,user_id,amount,total_amount,due_amount,due_date,status,created_at,date_created,loan_id,...,amt_paid_third_month,pct_paid_first_month,pct_paid_sec_month,pct_paid_third_month,amt_defaulted_till_first_month,amt_defaulted_till_sec_month,amt_defaulted_till_trd_month,amt_defaulted_first_month,amt_defaulted_sec_month,amt_defaulted_third_month
0,0,3070,6000.0,6045.28,6459000000,2022-05-02,repaid,2022-02-01 00:47:29.575000+00:00,2022-02-01,0.0,...,0.00,0.598707,1.024096,1.024096,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2546,6000.0,6045.28,6459000000,2022-05-02,repaid,2022-02-01 00:49:51.763000+00:00,2022-02-01,1.0,...,1698.75,0.319065,0.751914,1.032918,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2413,6000.0,6045.28,6459000000,2022-05-02,repaid,2022-02-01 01:24:40.537000+00:00,2022-02-01,2.0,...,3887.13,0.354946,0.354946,0.997949,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2585,6000.0,6045.28,6459000000,2022-05-02,debt_collection,2022-02-01 02:52:59.803000+00:00,2022-02-01,3.0,...,0.00,0.006948,0.026798,0.026798,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2556,6000.0,6045.28,6459000000,2022-05-02,repaid,2022-02-01 02:53:07.123000+00:00,2022-02-01,4.0,...,0.00,0.582590,0.582590,0.582590,3055.39,3055.39,3055.39,3055.39,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6741,6741,2130,2500.0,2518.87,3228820000,2023-01-01,repaid,2022-10-03 20:44:11.967000+00:00,2022-10-03,6741.0,...,0.00,1.028918,1.028918,1.028918,200.00,200.00,200.00,200.00,0.0,0.0
6742,6742,549,6000.0,6045.28,7749160000,2023-01-01,repaid,2022-10-03 21:03:02.995000+00:00,2022-10-03,6742.0,...,0.00,1.068086,1.068086,1.068086,NaN,NaN,NaN,NaN,NaN,NaN
6743,6743,1414,6000.0,6045.28,7749160000,2023-01-01,repaid,2022-10-03 21:33:47.964000+00:00,2022-10-03,6743.0,...,0.00,0.900476,1.056695,1.056695,1663.37,1663.37,1663.37,1663.37,0.0,0.0
6744,6744,2070,6000.0,6045.28,7749160000,2023-01-01,debt_repaid,2022-10-03 22:29:03.256000+00:00,2022-10-03,6744.0,...,1108.80,0.238798,0.320349,0.503765,16.59,16.59,16.59,16.59,0.0,0.0


### 3.4 - Create the most frequent form of repayment features

In [36]:
def calculate_most_frequent(
        dataframe:pd.DataFrame,
        col_to_count:str,
        col_to_group:str,
        return_counts:bool=False
)->pd.DataFrame:
    """
    Count the most frequent ocurrence of a value in a column in a dataframe.

    Args:
        dataframe (pd.DataFrame): Initial dataframe to count 
        col_to_count (str): Column to count the vaules.
        col_to_group (str): Column to group the vaules in dataframe.
        return_counts (bool, optional): If true, returns the counts of the most frequent value.
        If false, return only the group id and their most frequent value . Defaults to False.

    Returns:
        pd.DataFrame: A new dataframe with the most frequent value in the column by group. If
        return_counts is true, returns the frequency of the value per group. If return_counts
        is false, then, returns only the group and their most frequent value.
    """    
    df_counts_type = dataframe.groupby(by=col_to_group)[col_to_count]\
                              .value_counts()\
                              .to_frame(name="count_types")\
                              .reset_index()
    df_counts_type = df_counts_type.sort_values(by=[col_to_group,"count_types"],ascending=False)

    idxs_most_frequent = df_counts_type.groupby(by=col_to_group)["count_types"].idxmax()
    final_df = df_counts_type.loc[idxs_most_frequent,:].reset_index(drop=True)

    if return_counts:
        return final_df
    
    return final_df[[col_to_group,col_to_count]]

In [37]:
def calculate_most_frequent(
        dataframe:pd.DataFrame,
        col_to_count:Union[str,list],
        col_to_group:Union[str,list],
        return_counts:bool=False
)->pd.DataFrame:
    """
    Count the most frequent ocurrence of a value in a column in a dataframe.

    Args:
        dataframe (pd.DataFrame): Initial dataframe to count 
        col_to_count (str,list): Column to count the vaules.
        col_to_group (str,list): Column to group the vaules in dataframe.
        return_counts (bool, optional): If true, returns the counts of the most frequent value.
        If false, return only the group id and their most frequent value . Defaults to False.

    Returns:
        pd.DataFrame: A new dataframe with the most frequent value in the column by group. If
        return_counts is true, returns the frequency of the value per group. If return_counts
        is false, then, returns only the group and their most frequent value.
    """    
    df_counts_type = dataframe.groupby(by=col_to_group)[col_to_count]\
                              .value_counts()\
                              .to_frame(name="count_types")\
                              .reset_index()

    idxs_most_frequent = df_counts_type.groupby(by=col_to_group)["count_types"].idxmax()
    final_df = df_counts_type.loc[idxs_most_frequent,:].reset_index(drop=True)

    if return_counts:
        return final_df
    else:
         final_df = final_df.drop(columns="count_types")
    
    return final_df

In [38]:
dfs_all_repayments_per_loan = filter_amt_repayments_by_dates_per_loan_id(
    dataframe_loans=df_loans,
    dataframe_repayments=df_loans_repay,
    date_ref_col_df_loans="date_created",
    date_ref_col_df_repayments="date_created",
    begin_dates=first_month_begin_dates,
    end_dates=trd_month_end_dates
)

In [40]:
dfs_most_frequent_payment_per_id = [
    calculate_most_frequent(
        dataframe=df,
        col_to_count="type",
        col_to_group="loan_id"
    )
    for df in dfs_all_repayments_per_loan
]
df_most_frequent_payment_per_loan = pd.concat(dfs_most_frequent_payment_per_id)
df_most_frequent_payment_per_loan = df_most_frequent_payment_per_loan.drop_duplicates(subset=["loan_id"])
df_most_frequent_payment_per_loan = df_most_frequent_payment_per_loan.rename({"type":"most_frequent_repayment_type_for_loan"},axis=1)
df_most_frequent_payment_per_loan

,loan_id,most_frequent_repayment_type_for_loan
0,0,autopilot
1,1,autopilot
2,2,autopilot
3,3,autopilot
4,4,pix
...,...,...
26,6741,autopilot
27,6742,autopilot
28,6743,autopilot
29,6744,autopilot


In [41]:
df_loans_with_repay_feats = df_loans_with_repay_feats.merge(
    right=df_most_frequent_payment_per_loan.set_index(["loan_id"]),
    right_index=True,
    left_on="id",
    how="left"
)
print("Shape of final dataset:",df_loans_with_repay_feats.shape)
print("Columns of final dataset:",df_loans_with_repay_feats.columns)
df_loans_with_repay_feats

Shape of final dataset: (6746, 26)
Columns of final dataset: Index(['id', 'user_id', 'amount', 'total_amount', 'due_amount', 'due_date',
       'status', 'created_at', 'date_created', 'loan_id',
       'amt_paid_till_first_month', 'amt_paid_till_sec_month',
       'amt_paid_till_trd_month', 'amt_paid_first_month', 'amt_paid_sec_month',
       'amt_paid_third_month', 'pct_paid_first_month', 'pct_paid_sec_month',
       'pct_paid_third_month', 'amt_defaulted_till_first_month',
       'amt_defaulted_till_sec_month', 'amt_defaulted_till_trd_month',
       'amt_defaulted_first_month', 'amt_defaulted_sec_month',
       'amt_defaulted_third_month', 'most_frequent_repayment_type_for_loan'],
      dtype='object')


,id,user_id,amount,total_amount,due_amount,due_date,status,created_at,date_created,loan_id,...,pct_paid_first_month,pct_paid_sec_month,pct_paid_third_month,amt_defaulted_till_first_month,amt_defaulted_till_sec_month,amt_defaulted_till_trd_month,amt_defaulted_first_month,amt_defaulted_sec_month,amt_defaulted_third_month,most_frequent_repayment_type_for_loan
0,0,3070,6000.0,6045.28,6459000000,2022-05-02,repaid,2022-02-01 00:47:29.575000+00:00,2022-02-01,0.0,...,0.598707,1.024096,1.024096,NaN,NaN,NaN,NaN,NaN,NaN,autopilot
1,1,2546,6000.0,6045.28,6459000000,2022-05-02,repaid,2022-02-01 00:49:51.763000+00:00,2022-02-01,1.0,...,0.319065,0.751914,1.032918,NaN,NaN,NaN,NaN,NaN,NaN,autopilot
2,2,2413,6000.0,6045.28,6459000000,2022-05-02,repaid,2022-02-01 01:24:40.537000+00:00,2022-02-01,2.0,...,0.354946,0.354946,0.997949,NaN,NaN,NaN,NaN,NaN,NaN,autopilot
3,3,2585,6000.0,6045.28,6459000000,2022-05-02,debt_collection,2022-02-01 02:52:59.803000+00:00,2022-02-01,3.0,...,0.006948,0.026798,0.026798,NaN,NaN,NaN,NaN,NaN,NaN,autopilot
4,4,2556,6000.0,6045.28,6459000000,2022-05-02,repaid,2022-02-01 02:53:07.123000+00:00,2022-02-01,4.0,...,0.582590,0.582590,0.582590,3055.39,3055.39,3055.39,3055.39,0.0,0.0,pix
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6741,6741,2130,2500.0,2518.87,3228820000,2023-01-01,repaid,2022-10-03 20:44:11.967000+00:00,2022-10-03,6741.0,...,1.028918,1.028918,1.028918,200.00,200.00,200.00,200.00,0.0,0.0,autopilot
6742,6742,549,6000.0,6045.28,7749160000,2023-01-01,repaid,2022-10-03 21:03:02.995000+00:00,2022-10-03,6742.0,...,1.068086,1.068086,1.068086,NaN,NaN,NaN,NaN,NaN,NaN,autopilot
6743,6743,1414,6000.0,6045.28,7749160000,2023-01-01,repaid,2022-10-03 21:33:47.964000+00:00,2022-10-03,6743.0,...,0.900476,1.056695,1.056695,1663.37,1663.37,1663.37,1663.37,0.0,0.0,autopilot
6744,6744,2070,6000.0,6045.28,7749160000,2023-01-01,debt_repaid,2022-10-03 22:29:03.256000+00:00,2022-10-03,6744.0,...,0.238798,0.320349,0.503765,16.59,16.59,16.59,16.59,0.0,0.0,autopilot


## 5 - Save final table

In [42]:
df_loans_with_repay_feats.to_csv("./data/processed/loans_repayment_history_features.csv",index=False)